In [42]:
import pandas as pd
from langchain_community.document_loaders import SeleniumURLLoader
from tqdm.auto import tqdm
import time
import os

In [43]:
# https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_test.json
df_test = pd.read_json('raw/RuBQ_2.0_test.json')
# https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_paragraphs.json
df_prg = pd.read_json('raw/RuBQ_2.0_paragraphs.json')

# Вариант 1

In [ ]:
df_rag = []
for idx, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    try:
        if len(row['paragraphs_uids']['with_answer']) == 1:
            print(row['question_text'])
            print(row['answer_text'])
            p_id = row['paragraphs_uids']['with_answer'][0]
            prg = df_prg.query(f'uid == {p_id}')
            prg_text = prg['text'].values[0]
            prg_link = prg['ru_wiki_pageid'].values[0]
            print(prg_text)
            print('==================')
            print()
            
            fname = f'{prg_link}.txt'
            if not os.path.isfile(f"files/{fname}"):
                loader = SeleniumURLLoader(urls=[f'https://ru.wikipedia.org/?curid={prg_link}'])
                data = loader.load()
                content = data[0].page_content
                
                if len(content) > 0:
                    with open(f"files/{fname}", "w", encoding="utf-8") as f:
                        f.write(content)

            df_rag.append([row['question_text'], row['answer_text'], prg_text, f'{prg_link}.txt'])
    except:
        print(f'Ошибка: {idx}')
    
    time.sleep(1)
    
df_rag = pd.DataFrame(df_rag)
df_rag.columns = ['Вопрос','Правильный ответ','Контекст','Файл']

In [14]:
df_rag

,Вопрос,Правильный ответ,Контекст,Файл
0,Что может вызвать цунами?,Землетрясение,"Землетрясения, извержения вулканов и другие по...",19985.txt
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу,«Хижина дяди Тома» (англ. Uncle Tom's Cabin) —...,273921.txt
2,Кто создал Зимний дворец в Санкт-Петербурге?,Растрелли,Зимний дворец (в 1918—1943 годах — Дворец Иску...,50.txt
3,Какое современное государство занимает террито...,Ирак,"Нине́ви́я или Нин (аккад. 𒌷𒉌𒉡𒀀, Ninwe; ассир. ...",36300.txt
4,Как звали отца Северуса Снейпа?,Тоббиас Снегг (Снейп),Происхождение Снегга в основном стало известно...,172300.txt
...,...,...,...,...
919,На какой реке стоит Череповец?,Шексна,Пятый гидроузел — Шекснинский (Череповецкий) Ш...,42745.txt
920,Какая страна является победителем Чемпионата м...,Германия,Магдале́на Но́йнер (нем. Magdalena Neuner; род...,400782.txt
921,Кто спроектировал Mercedes-AMG GT?,"Роберт Лесник, Марк Фетерстон, Ян Кауль",Автомобиль Mercedes-Benz SLS AMG имеет длину в...,1989102.txt
922,Какова высота Биг Бена?,96,"Высота башни, включая шпиль — 96 метров (315 ф...",329717.txt


In [163]:
#df_rag.to_pickle('ru_rag_test_dataset.pkl')

# Вариант 2

In [14]:
import requests 
from bs4 import BeautifulSoup 

In [10]:
idx = '273921' 
page = requests.get(f'https://ru.wikipedia.org/?curid={idx}') 
page.status_code 

200

In [11]:
page.content

b'<!DOCTYPE html>\n<html class="client-nojs" lang="ru" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>\xd0\xa5\xd0\xb8\xd0\xb6\xd0\xb8\xd0\xbd\xd0\xb0 \xd0\xb4\xd1\x8f\xd0\xb4\xd0\xb8 \xd0\xa2\xd0\xbe\xd0\xbc\xd0\xb0 \xe2\x80\x94 \xd0\x92\xd0\xb8\xd0\xba\xd0\xb8\xd0\xbf\xd0\xb5\xd0\xb4\xd0\xb8\xd1\x8f</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xc2\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","\xd1\x8f\xd0\xbd\xd0\xb2\xd0\xb0\xd1\x80\xd1\x8c","\xd1\x84\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb0\xd0\xbb\xd1\x8c","\xd0\xbc\xd0\xb0\xd1\x80\xd1\x82","\xd0\xb0\xd0\xbf\xd1\x80\xd0\xb5\xd0\xbb\xd1\x8c","\xd0\xbc\xd0\xb0\xd0\xb9","\xd0\xb8\xd1\x8e\xd0\xbd\xd1\x8c","\xd0\xb8\xd1\x8e\xd0\xbb\xd1\x8c","\xd0\xb0\xd0\xb2\xd0\xb3\xd1\x83\xd1\x81\xd1\x82","\xd1\x81\xd0\xb5\xd0\xbd\xd1\x82\xd1\x8f\xd0\xb1\xd1\x80\xd1\x8c","\xd0\xbe\xd0\xba\xd1\x82\xd1\x8f\xd0\xb1\xd1\x80\xd1\x8

In [22]:
list(soup.children)
soup = BeautifulSoup(page.content, 'html.parser') 
soup.find_all('p')
soup.find_all('p')[0].get_text() 
print(soup.find('p').prettify())

<p>
 «
 <b>
  Хижина дяди Тома, или Жизнь среди низших
 </b>
 » (
 <a href="/wiki/%D0%90%D0%BD%D0%B3%D0%BB%D0%B8%D0%B9%D1%81%D0%BA%D0%B8%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA" title="Английский язык">
  англ.
 </a>
 <span lang="en" style="font-style:italic;">
  Uncle Tom's Cabin; or, Life Among the Lowly
 </span>
 ) —
 <a class="mw-redirect" href="/wiki/%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD_(%D0%B6%D0%B0%D0%BD%D1%80)" title="Роман (жанр)">
  роман
 </a>
 <a href="/wiki/%D0%91%D0%B8%D1%87%D0%B5%D1%80-%D0%A1%D1%82%D0%BE%D1%83,_%D0%93%D0%B0%D1%80%D1%80%D0%B8%D0%B5%D1%82" title="Бичер-Стоу, Гарриет">
  Гарриет Бичер-Стоу
 </a>
 1852 года, направленный против
 <a href="/wiki/%D0%A0%D0%B0%D0%B1%D1%81%D1%82%D0%B2%D0%BE_%D0%B2_%D0%A1%D0%A8%D0%90" title="Рабство в США">
  рабовладения в США
 </a>
 . Роман произвёл большой общественный резонанс; по некоторым оценкам, настолько обострил один из местных конфликтов на почве рабства, что он вылился в
 <a href="/wiki/%D0%93%D1%80%D0%B0%D0%B6%D0%B4%D0%B0%D0%BD%D1%8

# Вариант 3

In [28]:
import wikipedia
wikipedia.set_lang('ru')

In [51]:
df_rag = []
for idx, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    try:
        if len(row['paragraphs_uids']['with_answer']) == 1:
            print(row['question_text'])
            print(row['answer_text'])
            p_id = row['paragraphs_uids']['with_answer'][0]
            prg = df_prg.query(f'uid == {p_id}')
            prg_text = prg['text'].values[0]
            pageid = prg['ru_wiki_pageid'].values[0]
            print(prg_text)
            print('==================')
            print()
            
            fname = f'{pageid}.txt'
            if not os.path.isfile(f"files/{fname}"):
                article = wikipedia.page(pageid=pageid)
                content = article.content
                
                if len(content) > 0:
                    with open(f"files/{fname}", "w", encoding="utf-8") as f:
                        f.write(content)

            df_rag.append([row['question_text'], row['answer_text'], prg_text, fname])
    except:
        print(f'Ошибка: {idx}')
        
    time.sleep(3)
    
df_rag = pd.DataFrame(df_rag)
df_rag.columns = ['Вопрос','Правильный ответ','Контекст','Файл']

  0%|          | 0/2330 [00:00<?, ?it/s]

Что может вызвать цунами?
Землетрясение
Землетрясения, извержения вулканов и другие подводные взрывы (в том числе взрывы подводных ядерных устройств), оползни, ледники, метеориты и другие разрушения выше или ниже уровня воды — всё это обладает достаточным потенциалом, чтобы вызвать цунами. Первое предположение о том, что цунами связано с подводными землетрясениями, было высказано древнегреческим историком Фукидидом.

Кто написал роман «Хижина дяди Тома»?
Г. Бичер-Стоу
«Хижина дяди Тома» (англ. Uncle Tom's Cabin) — роман Гарриет Бичер-Стоу 1852 года, направленный против рабовладения в Америке. Роман произвёл большой общественный резонанс; по некоторым оценкам, настолько обострил один из местных конфликтов на почве рабства, что он вылился в Гражданскую войну в США.

Кто создал Зимний дворец в Санкт-Петербурге?
Растрелли
Зимний дворец (в 1918—1943 годах — Дворец Искусств) — главный императорский дворец России, расположенный по адресу: Дворцовая площадь, 2 / Дворцовая набережная, 38, город

Как звали коня Александра Македонского?
Буцефал
Ребёнок был очень вспыльчивым и чувствительным, но при этом проявлял любознательность и прилежание. В источниках есть целый ряд историй, показывающих незаурядные способности юного царевича. Так, однажды Александр принял персидских послов в отсутствие отца и «не задал ни одного детского или малозначительного вопроса, а расспрашивал о протяжённости дорог, о способах путешествия в глубь Персии, о самом царе — каков он в борьбе с врагами, а также о том, каковы силы и могущество персов»; удивлённые послы увидели в мальчике «величие замыслов и стремлений». В другой раз Александр смог объездить неукротимого коня Буцефала, поскольку понял, что тот просто боится собственной тени. После этого Филипп, согласно Плутарху, воскликнул: «Ищи, сын мой, царство по себе, ибо Македония для тебя слишком мала!».

Как звали мать Александра Македонского?
Олимпиада
До IV века до н. э. Македония была небольшим и слабым царством, страдавшим от вторжений фракийцев и

Кто придумал Алису Селезневу?
Булычев
Али́са И́горевна Селезнёва — главная героиня цикла детских фантастических книг Кира Булычёва «Приключения Алисы» и их экранизаций и производных работ, таких, как «Тайна третьей планеты», «Гостья из будущего» и «Алиса знает, что делать!». Алиса — девочка, живущая в Москве в конце XXI века.

Кто сочинил "Танец с саблями"?
Хачатурян
Танец с саблями — музыкальное произведение Арама Хачатуряна, написанное для последнего действия балета «Гаянэ» (1942). Наиболее известное и узнаваемое произведение композитора, сочетающее в себе армянскую народную музыку с джазовыми элементами. По словам Тиграна Мансуряна, это «контрапункт армянского свадебного танца из Гюмри и настоящего американского саксофона, приведший к очень интересному и органичному синтезу».

На какой планете родился Люк Скайвокер?
Татуин
Родился в медицинском центре Полис-Масса в день создания Галактической Империи. После смерти его матери Люка отправили на Татуин к сводному брату Энакина Оуэну Ла

Кто такие «морские свиньи»?
Киты (подотряд зубатых китов)
Морские свиньи (лат. Phocoenidae) — семейство морских млекопитающих парвотряда зубатых китов. Ранее причислялось к семейству дельфиновых.

Какой стране принадлежит остров Мадейра, давший название крепкому вину мадере?
Португалии
О́стров Маде́йра (порт. Ilha da Madeira) — главный остров одноимённого архипелага, находящегося в северной части Атлантического океана у северо-западного побережья Африки. Хотя остров расположен на Африканской литосферной плите, он уже почти шесть веков этнически, культурно, политически и экономически вполне относится к Европе. Вместе с другим заселённым островом архипелага Порту-Санту, а также островами Ильяш-Дезерташ и Селваженш образует Автономный регион Мадейры со столицей в городе Фуншале. Принадлежит Португалии.

Кто автор строк: "Белеет парус одинокий..."?
М.Ю.Лермонтов
«Па́рус» («Белеет парус одинокой…») — стихотворение, написанное 18-летним Михаилом Лермонтовым в Петербурге. Хрестоматийный приме

...

С какого года вручается Нобелевская премия?
1901
Первый Нобелевский банкет состоялся 10 декабря 1901 года одновременно с первым вручением премии.

Какова протяженность Садового кольца?
15.6
Садо́вое кольцо́, также Кольцо́ Б — круговая магистральная улица в центре Москвы. Длина — 15,6 км, ширина — 60—70 м.

В каком месяце был запущен первый советский спутник Земли?
4 октября 1957 года
Пуск состоялся 15 мая в 19:00 по местному времени. Старт прошёл нормально, но на 98-й секунде полёта произошёл сбой в работе одного из боковых двигателей, ещё через 5 секунд все двигатели автоматически отключились, и ракета упала в 300 км от старта. Причиной аварии было возникновение пожара в результате разгерметизации топливной коммуникации высокого давления. Вторая ракета, Р-7 № 6Л, была подготовлена с учётом полученного опыта, но запустить её вовсе не удалось. 10—11 июня делались многократные попытки пуска, но в последние секунды срабатывала защитная автоматика. Выяснилось, что причиной была неправильна

Где происходит оплодотворение яйцеклетки у женщины?
В маточных трубах
Оплодотворение происходит в маточной трубе. При слиянии яйцеклетки и сперматозоида образуется новая клетка — зигота, которая в течение 3-4 дней перемещается по маточной трубе в сторону матки. Перемещение эмбриона по маточной трубе происходит благодаря току трубной жидкости (за счёт биения ресничек стенки трубы и перистальтическим сокращениям мышц). Спустя 26-30 часов после оплодотворения зигота начинает делиться и образует новый многоклеточный эмбрион. Спустя двое суток после оплодотворения эмбрион состоит из 4 клеток, спустя 3 суток — из 8 клеток, спустя 4 суток — из 10-20 клеток, спустя 5 суток — из нескольких десятков клеток. Процесс деления яйцеклетки называют «дроблением», потому что размер эмбриона не увеличивается, а каждая дочерняя клетка уменьшается в размере.

Где сформировался импрессионизм как направление?
во Франции в
Импрессиони́зм (фр. impressionnisme ← impression «впечатление») — одно из крупнейших те

Какого роста был Петр 1?
2,03 м
Ещё ребёнком Пётр поражал людей красотой и живостью своего лица и фигуры. Из-за своего высокого роста — 203 см (6 футов 8 дюймов) — он выдавался в толпе на целую голову. В то же время, при таком большом росте, он был не богатырского сложения — носил обувь 39 размера, а одежду 48 размера. Руки Петра были также небольшие, и его плечи узкие для его роста, то же самое, его голова была также мала по сравнению с телом.

Какое издательство выпускало сатирический журнал Крокодил?
Рабочая газета
В 1922—1926 годах активно сотрудничал с «Известиями», в 1926—1929 годах — с «Комсомольской правдой». Печатался в журналах «Новый мир», «Молодая гвардия», «Огонёк», «Крокодил», «Красная нива» и др.

Кто изобрел радио?
Гульельмо Маркони, Александр Степанович Попов, Реджинальд Обри Фессенден, Уильям Дюбилье
Создателем первой успешной системы обмена информацией с помощью радиоволн (радиотелеграфии) на западе считается итальянский инженер Гульельмо Маркони (1895).

Что содержи

Кто является основателем города Самара?
Григорий Засекин
Князь Григо́рий О́сипович Засе́кин (ок.1550 — не ранее 1597) — государственный деятель, первый воевода городов-крепостей на Волге — Самары (1586), Переволока (1589) и Саратова (1590).

Кто является основателем Владивостока?
Николай Николаевич Муравьёв-Амурский
В течение продолжительного времени российское правительство искало опорный пункт на Дальнем Востоке; эту роль поочерёдно выполняли Охотск, Аян, Петропавловск-Камчатский, Николаевск-на-Амуре. К середине XIX века поиски форпоста зашли в тупик: ни один из портов не отвечал необходимому требованию: иметь удобную и защищённую гавань, рядом с торговыми путями. Силами генерал-губернатора Восточной Сибири Николая Муравьёва-Амурского был заключён Айгунский договор, началось активное исследование Приамурья, и позже, в результате подписания Тяньцзиньского и Пекинского трактатов, к России были присоединены в том числе территории современного Владивостока. Само название Владивосток появ

На какой реке стоит Череповец?
Шексна
Пятый гидроузел — Шекснинский (Череповецкий) Шлюзы № 7 (напор 12,2 м; камера 266 × 17,8 м) и № 8 (вторая нитка; камера 310 × 21,5 м). Шекснинская ГЭС мощностью 84 МВт, — на южном склоне в 50 км выше Череповца у посёлка Шексна.На северном склоне трасса пути частично совпадает с руслом реки Вытегра и проходит по водохранилищам, образованным гидроузлами. Водораздельный бьеф тянется от Пахомовского гидроузла на Вытегре до Череповецкого гидроузла на Шексне. Судоходная трасса здесь проходит по водораздельному каналу длиной 40 км (от Пахомовского гидроузла до посёлка Анненский Мост).

Какая страна является победителем Чемпионата мира по биатлону 2011?
Германия
Магдале́на Но́йнер (нем. Magdalena Neuner; род. 9 февраля 1987 года, Гармиш-Партенкирхен, Бавария, ФРГ) — немецкая биатлонистка, одна из самых титулованных спортсменок в истории этого вида спорта. Двукратная олимпийская чемпионка 2010 года, 12-кратная чемпионка мира (рекорд среди женщин), трёхкратна

In [52]:
df_rag

,Вопрос,Правильный ответ,Контекст,Файл
0,Что может вызвать цунами?,Землетрясение,"Землетрясения, извержения вулканов и другие по...",19985.txt
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу,«Хижина дяди Тома» (англ. Uncle Tom's Cabin) —...,273921.txt
2,Кто создал Зимний дворец в Санкт-Петербурге?,Растрелли,Зимний дворец (в 1918—1943 годах — Дворец Иску...,50.txt
3,Какое современное государство занимает террито...,Ирак,"Нине́ви́я или Нин (аккад. 𒌷𒉌𒉡𒀀, Ninwe; ассир. ...",36300.txt
4,Как звали отца Северуса Снейпа?,Тоббиас Снегг (Снейп),Происхождение Снегга в основном стало известно...,172300.txt
...,...,...,...,...
917,На какой реке стоит Череповец?,Шексна,Пятый гидроузел — Шекснинский (Череповецкий) Ш...,42745.txt
918,Какая страна является победителем Чемпионата м...,Германия,Магдале́на Но́йнер (нем. Magdalena Neuner; род...,400782.txt
919,Кто спроектировал Mercedes-AMG GT?,"Роберт Лесник, Марк Фетерстон, Ян Кауль",Автомобиль Mercedes-Benz SLS AMG имеет длину в...,1989102.txt
920,Какова высота Биг Бена?,96,"Высота башни, включая шпиль — 96 метров (315 ф...",329717.txt


In [56]:
import glob 
all_files = glob.glob('files/*')
all_files = [f.split('\\')[-1] for f in all_files]
all_files

['100141.txt',
 '101.txt',
 '101560.txt',
 '10222.txt',
 '10223.txt',
 '10292.txt',
 '10300.txt',
 '103043.txt',
 '1032522.txt',
 '103353.txt',
 '1038.txt',
 '1062192.txt',
 '10647.txt',
 '10658.txt',
 '10669.txt',
 '10670.txt',
 '106717.txt',
 '1068248.txt',
 '1068936.txt',
 '1069005.txt',
 '106970.txt',
 '107225.txt',
 '107304.txt',
 '107694.txt',
 '107723.txt',
 '1077294.txt',
 '107928.txt',
 '108207.txt',
 '108301.txt',
 '1084.txt',
 '109038.txt',
 '109211.txt',
 '10947.txt',
 '1097526.txt',
 '110.txt',
 '11028.txt',
 '110316.txt',
 '110520.txt',
 '111255.txt',
 '111393.txt',
 '111408.txt',
 '1118275.txt',
 '1121.txt',
 '1125611.txt',
 '11314.txt',
 '11377.txt',
 '11378.txt',
 '113796.txt',
 '1142251.txt',
 '11514.txt',
 '11525.txt',
 '11542.txt',
 '118103.txt',
 '118505.txt',
 '11987.txt',
 '1210101.txt',
 '121720.txt',
 '122720.txt',
 '122991.txt',
 '1234185.txt',
 '124402.txt',
 '1247934.txt',
 '1248659.txt',
 '124940.txt',
 '1261274.txt',
 '126868.txt',
 '1275460.txt',
 '128351

In [30]:
idx = 743
article = wikipedia.page(pageid=idx)
print(article.content)

Зо́лото (химический символ — Au, от лат. Aurum) — элемент 11 группы (по устаревшей классификации — побочной подгруппы первой группы) шестого периода периодической системы химических элементов с атомным номером 79.
Простое вещество золото — благородный металл жёлтого цвета.


== Этимология ==
Русское слово «золото» восходит к праславянскому *zolto; родственны ему лит. geltonas «жёлтый», латыш. zeltais «золотой»; с другим вокализмом: готск. gulþ, нем. Gold, англ. gold; далее — санскр. हिरण्य (IAST: híraṇya), венг. arany, авест. zaranya, осет. zærījnæ «золото», также санскр. हरि (IAST: hari) «жёлтый, золотистый, зеленоватый», происходящих от праиндоевропейского корня *ǵʰel- «жёлтый, зелёный, яркий».


== Физические свойства ==
Чистое золото — мягкий металл, из-за релятивистских эффектов электронов имеющий жёлтый цвет. Красноватый оттенок некоторым изделиям из золота, например, монетам, придают примеси других металлов, в частности, меди. В тонких плёнках золото просвечивает зелёным. Золото